In [1]:
import os

# Set env vars BEFORE importing huggingface modules
os.environ["HF_HOME"] = "/projects/sciences/computing/sheju347/.cache/huggingface"
os.environ["TRANSFORMERS_CACHE"] = "/projects/sciences/computing/sheju347/.cache/huggingface/hub"

# Now import huggingface modules
from huggingface_hub import constants

print("HF_HOME:", constants.HF_HOME)
print("HF_HUB_CACHE:", constants.HF_HUB_CACHE)

HF_HOME: /projects/sciences/computing/sheju347/.cache/huggingface
HF_HUB_CACHE: /projects/sciences/computing/sheju347/.cache/huggingface/hub


In [2]:
import sys
sys.path.append("../")

In [3]:
# file_name = "../10-3-test_training_data.txt"
file_name_list = ["10-2-MedQA_context_training.txt", 
                  # "10-8-MedQA_Evol_context_training.txt"
                 ]

class TrainingData:
    def __init__(self):
        self.question_and_options = ""
        self.doc_list = []
        self.expected_output = ""

training_data_list = []

for file_name in file_name_list:
    with open(file_name) as f:
        question_index = None
        is_reading_question = False
        is_reading_answer = False
        current_training_data = None
        for line in f:
            key = "[question index:]"
            start = line.find(key)
            if start != -1:
                # print(question)
                
                question_index = int(line[start + len(key):].strip())
                # print(question_index)
    
                current_training_data = TrainingData()
                training_data_list.append(current_training_data)
                
                is_reading_question = False
                is_reading_answer = False
                continue
                
            key = "[question_and_options:]"
            start = line.find(key)
            if start != -1:
                current_training_data.question_and_options += line[start + len(key):]
                is_reading_question = True
                continue
    
    
            key = "[rag_doc:]"
            start = line.find(key)
            if start != -1:
                doc_idx_and_doc = line[start + len(key):]
                split_pos = doc_idx_and_doc.find(":")
                doc_idx = int(doc_idx_and_doc[:split_pos])
                doc = doc_idx_and_doc[split_pos + 1:]
                current_training_data.doc_list.append(doc)
                
                is_reading_question = False
                continue
    
            key = "[expected_output:]"
            start = line.find(key)
            if start != -1:
                expected_output = line[start + len(key):]
                current_training_data.expected_output += expected_output
                is_reading_answer = True
                continue
            
            if is_reading_question:
                current_training_data.question_and_options += line
                continue
                
            if is_reading_answer:
                current_training_data.expected_output += line
                continue
                
            # if "[rag_doc:]" in line:
            #     doc = line(

print(len(training_data_list))

if len(training_data_list[len(training_data_list) - 1].doc_list) == 0:
    del training_data_list[len(training_data_list) - 1]

print(len(training_data_list))
# print("------------")
# print(training_data_list[0].question_and_options)
# print("------------")
# print(len(training_data_list[0].doc_list))
# print("------------")
# print(training_data_list[0].doc_list[0])
# print("------------")
# print(training_data_list[0].expected_output)


9343
9343


In [4]:
from model_trainer import ModelTrainer
trainer = ModelTrainer()
# model_name = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/fine_tuned_model_entire_UltraMedical_batch_4"
# model_name = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/base/10-15-UltraMedical-batchsize8-bf16"
# model_name = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/base_qwen/11-11-Qwen3-4B-base-UltraMedical"
# model_name = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/base/12-1-phi3-mini-batchsize8-epoch456/checkpoint-256000"
model_name = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/base/12-9-phi3.5-mini-UltraMedical-batchsize8-epoch456/checkpoint-256000"
# model_name = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/base_qwen/11-25-qwen-4B-Thinking-batch8-epoch456/checkpoint-204800"

# model_name = "TsinghuaC3I/Llama-3.1-8B-UltraMedical"

tokenizer_path = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/base/12-9-phi3.5-mini-UltraMedical-batchsize8-epoch456/"
# tokenizer_path = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/base_qwen/11-25-qwen-4B-Thinking-UltraMedical"
trainer.load_model(model_name, tokenizer_path = tokenizer_path)

tokenizer = trainer.tokenizer
model = trainer.model

/projects/sciences/computing/sheju347/miniconda3/envs/LLM311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/projects/sciences/computing/sheju347/miniconda3/envs/LLM311/lib/python3.11/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


---------- start checking GPU -----------
GPU: NVIDIA H100 NVL
torch.cuda.is_bf16_supported():  True
---------- finish checking GPU -----------
---------- start loading model:/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/base/12-9-phi3.5-mini-UltraMedical-batchsize8-epoch456/checkpoint-256000 -----------
finish loading tokenizer


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  4.72it/s]


finish loading model
torch_dtype: torch.bfloat16
cuda available: True
device: cuda
---------- finish loading model:/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/base/12-9-phi3.5-mini-UltraMedical-batchsize8-epoch456/checkpoint-256000 -----------


In [5]:

def convert_to_chat_format(conversations, is_input):
    chat = []
    for conversation in conversations:
        if conversation["from"] == "human":
            value = conversation["value"]
            for option_letter in ["A", "B", "C", "D"]:
                value = value.replace(f"{option_letter}.", f"[{option_letter}] :")
            chat.append({"role": "user", "content": value})
        elif is_input == False and conversation["from"] == "gpt":
            value = conversation["value"]
            for option_letter in ["A", "B", "C", "D"]:
                value = value.replace(f"{option_letter}.", f"[{option_letter}].")
            chat.append({"role": "assistant", "content": value})
    return chat

In [6]:
import torch

max_length = 2048 # 2048
system_prompt = ""
# system_prompt = "You are a medical question answering assistant."

def convert_to_chat_format(question, context, expected_output, is_input):
    chat = []
    if system_prompt:
        chat.append({"role": "system", "content": system_prompt})
    
    content = ""
    if context:
        content += f"Context:\n {context}\n\nQuestion: \n"
    content += question
    chat.append({"role": "user", "content": content})

    if is_input == False:
        chat.append({"role": "assistant", "content": expected_output})
                     
    return chat

def convert_to_tokenized_training_data(question, context, expected_output):
    questions = convert_to_chat_format(question, context, expected_output, is_input = True) #[convert_to_chat_format(conversations, is_input = True) for conversations in conversations_list]   
    template_questions = tokenizer.apply_chat_template(questions, tokenize = False, add_generation_prompt = True)
    
    questions_and_answers = convert_to_chat_format(question, context, expected_output, is_input = False) #[convert_to_chat_format(conversations, is_input = False) for conversations in conversations_list]
    template_questions_and_answers = tokenizer.apply_chat_template(questions_and_answers, tokenize = False, add_generation_prompt = False)

    # print("template_questions", '\n', template_questions, '\n------')
    # print("template_expected_outputs", '\n', template_questions_and_answers, '\n-----')

    tokenized_questions = tokenizer(template_questions, padding = False, truncation=True, max_length=max_length)
    tokenized_inputs = tokenizer(template_questions_and_answers, padding=False, truncation=True, max_length=max_length)

    # print("tokenized_questions", '\n', tokenized_questions, '\n------')
    # print("tokenized_inputs", '\n', tokenized_inputs, '\n-----')

    questions_input_ids = tokenized_questions["input_ids"]
    all_content_input_ids = tokenized_inputs["input_ids"]
    tokenized_labels = [-100] * len(questions_input_ids) + all_content_input_ids[len(questions_input_ids):]

    # print("tokenized_labels", '\n', tokenized_labels, '\n-----')
    
    # Add padding
    padding_num = max_length - len(tokenized_inputs["input_ids"])
    if padding_num > 0:
        tokenized_inputs["input_ids"] = [tokenizer.eos_token_id] * padding_num + tokenized_inputs["input_ids"]
        tokenized_inputs["attention_mask"] = [0] * padding_num + tokenized_inputs["attention_mask"]
        tokenized_labels = [tokenizer.eos_token_id] * padding_num + tokenized_labels

    # print(tokenized_inputs)
    # print(len(tokenized_inputs["input_ids"]), len(tokenized_inputs["attention_mask"]), len(tokenized_inputs["labels"]))

    training_data = {"input_ids": tokenized_inputs["input_ids"], 
                    "attention_mask": tokenized_inputs["attention_mask"],
                    "labels": tokenized_labels}
    
    # print("input_ids:", len(training_data["input_ids"]), training_data["input_ids"])

    # print("attention_mask:", len(training_data["attention_mask"]), training_data["attention_mask"])

    # print("labels:", len(training_data["labels"]), training_data["labels"])


    training_data = {k: torch.tensor(v) for k, v in training_data.items()}

    # print("training_data:", training_data)
    
    return training_data

In [7]:
import random

doc_num_list = [0, 1, 2]
# doc_num_list = [2]
is_random_doc_num = False

all_training_data = []

def get_tokenized_data(data, doc_num):
    context = ""
    for j in range(doc_num):
        context += data.doc_list[j]
        context += "\n\n"
    tokenized_data = convert_to_tokenized_training_data(data.question_and_options, context, data.expected_output)
    return tokenized_data

for i in range(len(training_data_list)):
    data = training_data_list[i]

    # n docs
    if is_random_doc_num:
        doc_num = random.choice(doc_num_list)
        all_training_data.append(get_tokenized_data(data, doc_num))
    else:
        for doc_num in doc_num_list:
            all_training_data.append(get_tokenized_data(data, doc_num))

    if i % 1000 == 0:
        print(i)

    # if len(all_training_data) > 10:
    #     break
        
from datasets import Dataset

training_data = Dataset.from_list([
    {
        "input_ids": s["input_ids"].tolist(),
        "attention_mask": s["attention_mask"].tolist(),
        "labels": s["labels"].tolist(),
    }
    for s in all_training_data
])

print(training_data)

# ret = convert_to_tokenized_training_data(data.question_and_options, data.doc_list[0], data.expected_output)
# print(ret)
# print(len(ret["input_ids"]))
# print(len(ret["attention_mask"]))
# print(len(ret["labels"]))

# print(ret["input_ids"].tolist())
# print(ret["attention_mask"].tolist())
# print(ret["labels"].tolist())

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 28029
})


In [8]:
sample = training_data[2]
print(sample.keys())

import torch
labels = sample["labels"]
# print(torch.unique(labels))  # should include token IDs, not only -100

print(len(sample["labels"]))
print(tokenizer.decode(sample["input_ids"], skip_special_tokens = True))
print("--------------")

# print(tokenizer.decode(sample["labels"]))
# print(tokenizer.decode([id for id in sample["labels"] if id >= 0], skip_special_tokens = True))

dict_keys(['input_ids', 'attention_mask', 'labels'])
2048
Context:
 [Acute cystourethritis during pregnancy]. A prospective study was carried out in 103/863 obstetric patients with cystitis characterized by urinary urgency and frequency, dysuria, pyuria and suprapubic discomfort in the absence of systemic symptoms such as fever and costovertebral angle tenderness. The association of symptomatic lower urinary tract infection with low-count bacteriuria (10(2)-10(5) UFC/mL of urine) was present in all the patients. The incidence of cystourethritis was about 12%, most of the infections occurred at the first trimester. To learn whether a multiple-dose of nitrofurantoin or ampicillin is safe and effective therapy for acute uncomplicated urinary tract infections, 103 symptomatic pregnant women were randomly grouped to receive oral nitrofurantoin (100 mg t.i.d.) or ampicillin (500 mg t.i.d.) for five days. Seventeen patient were excluded since they did not return for follow-up. Escherichia col

In [ ]:
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# model.gradient_checkpointing_enable() # TODO

use_lora = True
if use_lora:
    
    model.config.name_or_path = model_name
    model.name_or_path = model_name
    
    lora_config = LoraConfig(
        r=16,                    # rank (typical 8–64)
        lora_alpha=32,           # scaling factor
        # target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # for Phi-3     (attention projection layers)
        target_modules=["qkv_proj", "o_proj"], # for Phi-3.5
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )
    
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()


# output_dir = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/LoRA/11-10-LoRA-5e-5-r16-context012"
# output_dir = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/LoRA_qwen/11-14-LoRA-1e-4-r16-context012"
# output_dir = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/LoRA/12-6-epoch5-model-LoRA-1e-4-r16-context012"
# output_dir = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/LoRA/12-9-epoch5-model-LoRA-1e-4-r16-context0123"
# output_dir = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/LoRA_qwen/12-8-Qwen-4B-Thinking-epoch4-LoRA-1e-4-r16-context012"
output_dir = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/LoRA/12-13-Phi3.5-epoch5-lora-1e-4-r16-context012"

is_bf16_supported = torch.cuda.is_bf16_supported()

training_args = TrainingArguments(
    output_dir = output_dir,
    save_strategy = "epoch", #"steps", #"steps", # save checkpoints # "epoch", "steps", "no"
    # save_steps = 50000, # total_steps = dataset_size / batch_size
    # save_total_limit = 5, # keep only the last N checkpoint
    per_device_train_batch_size = 4, # As specified in the paper: batch_size: 32
    num_train_epochs = 3, # As specified in the paper: 3 epochs
    learning_rate = 1e-4, # As specified in the paper: 1e-4
    fp16 = torch.cuda.is_available() and not is_bf16_supported,
    bf16 = is_bf16_supported, # Sometimes RuntimeError: "_amp_foreach_non_finite_check_and_unscale_cuda" not implemented for 'BFloat16'
    logging_steps = 100,
    logging_dir = "./logs",  # Directory for logs
    # report_to = ["tensorboard"],  # Enable logging to TensorBoard
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = training_data
)

trainer.train()

if not use_lora:
    model.to(torch.bfloat16)  # Convert to bfloat16
    
model.save_pretrained(output_dir, 
                        # save_function=torch.save,  # Use standard PyTorch save
                        # state_dict=model.state_dict(),  # Only save the model weights
                        # safe_serialization=True,  # More efficient serializationsave_optimizer_state=False
                    )
tokenizer.save_pretrained(output_dir,
                        # legacy_format=False  # Use newer, more efficient format
                    )

print(f"----------- finish saving model {model.name_or_path} to: {output_dir} -----------")

trainable params: 9,437,184 || all params: 3,830,516,736 || trainable%: 0.2464


Step,Training Loss
100,0.110400
200,0.109700
300,0.105700
400,0.101800
500,0.098800
600,0.101300
700,0.105200
800,0.101600
900,0.099100
1000,0.104100
